# How to build Angular components?

How to create a mock filesystem?


In [1]:
var execSync = require('child_process').execSync;
try {
    require.resolve('memory-fs');
} catch (e) {
    execSync('npm install memory-fs');
}

// first install angular cli
var path = require('path');
var MemoryFileSystem = require("memory-fs");
var data = {
    'angular-cli.json': 
`
{
    "project": {
        "name": "act-ecommerce"
    },
    "apps": [
        {
            "root": "src",
            "main": "main.ts",
            "tsconfig": "../tsconfig.json",
            "prefix": "bc",
            "styles": [],
            "scripts": []
        }
    ]
}
`
};
var memfs = new MemoryFileSystem(data);
(data);  // return blueprint



{ 'angular-cli.json': '\n{\n    "project": {\n        "name": "act-ecommerce"\n    },\n    "apps": [\n        {\n            "root": "src",\n            "main": "main.ts",\n            "tsconfig": "../tsconfig.json",\n            "prefix": "bc",\n            "styles": [],\n            "scripts": []\n        }\n    ]\n}\n' }

Run angular-cli commands from nodejs?

Add a component to the specified project using angular-cli?



In [2]:
var execSync = require('child_process').execSync;
try {
    require.resolve('@angular/cli');
} catch (e) {
    execSync('npm install @angular/cli');
}
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;

// use utility/filesystem to mock all fs and typescript commands
var mockTypescriptFs, mock;

var importer = require('../Core');
// call the CLI just like from command line
var ng = (project, args = ['generate', 'component', 'test']) => {
    // set up project path
    var previous = process.cwd();
    
    var conf = {
        cliArgs: args,
        inputStream: process.stdin,
        outputStream: process.stdout
    };
    
    // execute
    process.chdir(project);
    return importer.interpretAll('memory-fs rewire')
        .then(r => {
            mockTypescriptFs = eval("'use strict';" + r[0].code);
            mock = mockTypescriptFs(memfs);
    
            // overlay out temp filesystem on top of current filesystem
            memfs.mkdirpSync(project);
            memfs.writeFileSync(
                path.join(project, 'angular-cli.json'),
                data['angular-cli.json']);
                
            var cli = require('@angular/cli');
            return cli(conf);
        })
        // ng is mocked at this point
        .then(r => {
            process.chdir(previous);
            mock.stopAll();
            return data;
        });
}
(ng);



[Function: ng]

List projects by name?



In [3]:
var execSync = require('child_process').execSync();
try {
    require.resolve('glob');
} catch (e) {
    execSync('glob');
}
var glob = require('glob');

// But we also want to automatically load projects?
var listProjects = (root = PROFILE_PATH) => {
    return new Promise((resolve, reject) => {
        glob('**/package.json', {
            ignore: ['**/node_modules/**',
                    '**/packages/**',
                    '**/vendor/**',
                    '/Users/**/Downloads/**',
                    '/Users/**/Library/**',
                    '/Users/**/Applications/**',
                    '/Users/**/Music/**',
                    '**/wp-content/**',
                    '**/wp-includes/**',
                    '**/Pods/**',
                    '**/svn/**',
                    '**/.git/**',
                    '**/.vscode/**',
                    '**/.npm/**',
                    '**/Cache/**',
                    '**/Creative Cloud Files/**'],
            cwd: path.resolve(root),
            silent: true,
            nodir: true,
            strict: false
        }, (err, matches) => {
            var result = {};
            matches.forEach(m => {
                var projectPath = path.dirname(path.join(root, m));
                result[path.basename(projectPath)] = projectPath;
            })
            return resolve(result);
        });
    });
};
(listProjects);



[Function: listProjects]

Build angular modules from nodejs?



In [4]:
// Wouldn't it be nice if we could list these projects as interactive icons?
var fs = require('fs');
var convertNgUniversal = (project = '~/Documents/portal') => {
    return ng(project, ['generate', 'module', '--spec', '--flat', 'AppServer'])
    .then(() => {
        var serverModulePath = project + '/src/app/app-server.module.ts';
        var app_server = memfs.readFileSync(serverModulePath).toString();
        app_server = app_server.replace('CommonModule', `ServerModule,
AppModule
],
bootstrap: [
	  AppComponent
`);
        memfs.writeFileSync(serverModulePath, app_server);


        var appModulePath = project + '/src/app/app.module.ts';
        var app_module = fs.readFileSync(appModulePath).toString();
        app_module = app_module.replace('BrowserModule,',` BrowserModule.withServerTransition({
            appId: 'universal-demo-app'
        }),
`);
        return importer.getCells('./angular universal.ipynb')
    })
    .then(cells => {
        var appServerPath = project + '/src/main.server.ts';
        memfs.writeFileSync(appServerPath, cells.join('\n'));
    })
};
(convertNgUniversal);



[Function: convertNgUniversal]

Compile the project if needed?

In [5]:
var execSync = require('child_process').execSync;
try {
    require.resolve('webpack');
    require.resolve('@ngtools/webpack');
} catch (e) {
    execSync('npm install webpack @ngtools/webpack');
}

// source https://github.com/christianalfoni/webpack-bin/issues/106
var webpackAngularProject = (project, memfs) => {
    // set up project path
    var previous = process.cwd();
    process.chdir(project);
    module.paths.unshift(project + '/node_modules');
    
    var webpack = require('webpack');
    var ngtools = require('@ngtools/webpack');

    var conf = {
        entry: {
            main: './src/main.server.ts'
        },
        resolve: {
          extensions: ['.ts', '.js']
        },
        target: 'node',
        output: {
            path: process.cwd() + '/dist',
            filename: '[name].js'
        },
        plugins: [
            new ngtools.AotPlugin({
                tsConfigPath: process.cwd() + '/tsconfig.json',
            })
        ],
        module: {
            rules: [
                {
                  test: /\.ts$/,
                  loader: '@ngtools/webpack',
                }
            ]
        }
    }

    var compiler = webpack(conf);

    compiler.inputFileSystem = memfs;
    compiler.outputFileSystem = memfs;
    compiler.resolvers.normal.fileSystem = memfs;
    compiler.resolvers.context.fileSystem = memfs;

    return new Promise((resolve, reject) => {
        compiler.run(function(err, stats) {
            process.chdir(previous);
            if (err) reject(err);
            resolve(eval(stats.compilation.assets['main.js'].source()));
        });
    });
}
(webpackAngularProject);


[Function: webpackAngularProject]

Display angular modules in jupyter?



In [6]:
$$.async();

/*
            return listProjects(project);
        })
        .then(r => {
            project = r[path.basename(project)];
*/

var project = '~/Documents/portal';
project = project.replace(/^~\//ig, PROFILE_PATH + '\/');

convertNgUniversal(project)
//    .then(() => ng(project, ['build', '--aot', '--prod']))
    .then(() => webpackAngularProject(project, memfs))
    //.then(() => execCmd())
    .then(r => {
        console.log(r);
        var data = memfs.data;
        path.resolve(project)
            .split(path.sep)
            .filter((e, i) => i > 0 || e !== '')
            .forEach(p => data = data[p]);
        return data;
    })
    .catch(e => console.log(e))
    .then(r => $$.sendResult(r))


installing module
  create src/app/app-server.module.spec.ts
  create src/app/app-server.module.ts
  WARNING Module is generated but not provided, it must be provided to be used


Error: Module build failed: TypeError: Cannot read property 'length' of undefined
    at createSourceFile (/Users/briancullinan/Documents/portal/node_modules/typescript/lib/typescript.js:15457:109)
    at parseSourceFileWorker (/Users/briancullinan/Documents/portal/node_modules/typescript/lib/typescript.js:15389:26)
    at Object.parseSourceFile (/Users/briancullinan/Documents/portal/node_modules/typescript/lib/typescript.js:15338:26)
    at Object.createSourceFile (/Users/briancullinan/Documents/portal/node_modules/typescript/lib/typescript.js:15192:29)
    at new TypeScriptFileRefactor (/Users/briancullinan/Documents/portal/node_modules/@ngtools/webpack/src/refactor.js:29:35)
    at Object.ngcLoader (/Users/briancullinan/Documents/portal/node_modules/@ngtools/webpack/src/loader.js:340:26)
    at Object.eval (eval at <anonymous> (evalmachine.<anonymous>:57:21), <anonymous>:73:7)
    at __webpack_require__ (eval at <anonymous> (evalmachine.<anonymous>:57:21), <anonymous>:20:30)
    at 